In [7]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import pickle
from actsnclass.metrics import get_snpcc_metric
from shutil import copyfile

In [8]:
all_fom = []

for i in range(100):
    fname = 'results_42876/UncSampling/metrics/metrics_UncSampling_v' + str(i) + '.dat'
    data = pd.read_csv(fname, index_col=False, delim_whitespace=True)
    all_fom.append([i, data.iloc[59]['fom']])
    
all_fom = np.array(all_fom)

In [9]:
fom_mean = np.mean(all_fom[:,1])

In [10]:
distance =  abs(all_fom[ :,1] - fom_mean)
indx = np.argsort(distance)

In [11]:
distance[indx[0]]

1.1851814568086816e-05

In [12]:
indx[0]

59

In [13]:
all_fom[indx[0]]

array([59.        ,  0.27992963])

In [15]:
fname = 'results_42876/UncSampling/training_samples/final/finaltrain_v' + str(indx[0]) + '.csv'
data_train = pd.read_csv(fname, index_col=False)

fname = 'results_42876/UncSampling/test_samples/final/finaltest_v' + str(indx[0]) + '.csv'
data_test = pd.read_csv(fname, index_col=False)

In [16]:
clf = RandomForestClassifier(random_state=42, n_estimators=1000)
clf.fit(data_train[list(data_train.keys())[:-2]], data_train['type'].values == 'Ia')

RandomForestClassifier(n_estimators=1000, random_state=42)

In [17]:
clf.score(data_test[list(data_test.keys())[:-2]], data_test['type'].values == 'Ia')

0.9777834882960332

In [18]:
pred = clf.predict(data_test[list(data_test.keys())[:-2]])

In [19]:
get_snpcc_metric(pred, data_test['type'].values == 'Ia')

(['accuracy', 'efficiency', 'purity', 'fom'],
 [0.9777834882960332,
  0.49746192893401014,
  0.8313891834570519,
  0.3092864014942617])

In [77]:
pickle.dump(clf, open('results_42876/UncSampling/models/mean_model_42876.pkl', 'wb'))

### slight off topic - compute the final training and test sample for all runs

In [5]:
for i in range(100):
    
    fname = 'results_42876/UncSampling/training_samples/initial/initialtrain_v' + str(i) + '.csv'
    data_train_ini = pd.read_csv(fname, index_col=False)

    fname2 = 'results_42876/UncSampling/queries/queried_UncSampling_v' + str(i) + '.dat'
    data_queried = pd.read_csv(fname2, index_col=False, delim_whitespace=True)
    data_queried.rename(columns={'id': 'objectId'})

    fname3 = 'results_42876/UncSampling/test_samples/initial/initial_test_v' + str(i) + '.csv'
    data_test_ini = pd.read_csv(fname3, index_col=False)

    flag_queried = np.array([item  in data_queried['id'].values for item in data_test_ini['objectId'].values])
    data_test = data_test_ini[~flag_queried]
    data_test.to_csv('results_42876/UncSampling/test_samples/final/finaltest_v' + str(i) + '.csv', index=False)
    
    data_train = pd.concat([data_train_ini, data_test_ini[flag_queried]], ignore_index=True)
    data_train.to_csv('results_42876/UncSampling/training_samples/final/finaltrain_v' + str(i) + '.csv', index=False)
    